# Managing population consumption

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../../az-function/")

import yaml, ssl, asyncio, pickle, os, ast

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [2]:
os.listdir("../../az-function/app")

['configurations', 'connectors', 'functions', 'objects']

Much of the Population growth is handled in scripts inside the Azure Function

In [3]:
# mapping to the modules that make the app

from app.connectors import cmdb_graph
from app.functions import consumption
from app.objects import time

c = cmdb_graph.CosmosdbClient()


executing local windows deployment


In [4]:
params = yaml.safe_load(open(os.path.join(os.getenv("abspath"),"app/configurations/popgrowthconfig.yaml")))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'

Time is updated by the `time` function. It is always running.

In [5]:
t = time.Time(c)
t.get_current_UTU()
params['currentTime'] = t.params['currentTime']
t

< time at: 2024-05-31T20:52:33.234215+00:00 UTU:17428 >

In [6]:
params

{'pop_health_requirement': 0.7,
 'pop_consumes': 2,
 'starve_damage': 0.05,
 'changing_values': ['conformity',
  'literacy',
  'aggression',
  'constitution',
  'wealth',
  'factionLoyalty'],
 'currentTime': 17428}

For this notebook, setting the `pop_health_requirement` to an arbitrary value just to examine.

## Population Resource Consumption

I've given up on trying to get a list of planets with the consumption type. I'm convinced that this could be an Azure Cosmos issue that I can't solve. 

```
g.V().has('label','planet').
    project('planetName', 'grouping').
        by('name').
        by(in('inhabits').out('isOf').as('species').groupCount().by('consumes'))
```

So first we are going to get a list of the locations who have populations. It could be any location, including planets. So I'm going to query based on 

In [7]:
consuming_planets = consumption.get_consuming_planets(c)
consuming_planets

[{'name': 'Damshamunling',
  'label': 'planet',
  'objid': '6135767546801',
  'consumes': [{'organics': 7}]},
 {'name': 'Teinspringsgio',
  'label': 'planet',
  'objid': '9482778660790',
  'consumes': [{'organics': 7}]}]

Then we will create an action event for each pop

In [11]:
message = {"job":{},"agent":consuming_planets[0]}
message

{'job': {},
 'agent': {'name': 'Damshamunling',
  'label': 'planet',
  'objid': '6135767546801',
  'consumes': [{'organics': 7}]}}

## Looking more closely at starvation

* Grab my testing planet, with people on it
* Lower that amount of resource on that planet to 0
* run the lower health metric
* watch the people starve and die

In [47]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').valuemap()
    """)

planet = c.clean_node(c.res[0])
planet

{'name': 'Lecarrang',
 'class': 'terrestrial',
 'objid': '2423418653337',
 'radius': 0.54,
 'mass': 0.821,
 'orbitsDistance': 0.939,
 'orbitsId': '5618858323815',
 'orbitsName': 'Senpol',
 'isSupportsLife': 'true',
 'isPopulated': 'true',
 'isHomeworld': 'true',
 'username': 'Billmanserver',
 'objtype': 'planet',
 'id': '2423418653337'}

Grabbing that planets organics as well, and setting the amount of the organics to zero

In [48]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').has('name','Organic').property('volume',-1)
    """)

In [49]:
consumption_df.loc[consumption_df['location_id']==planet['objid'],['remaining','available']] = -1
consumption_df

,location_id,consumes,pop,consumption,multi,available,remaining
0,9003968629604,organics,12,24,False,1157.0,1133.0


In [50]:
username = 'Billmanserver'
c.run_query(f"""
            g.V().haslabel('planet').has('username','{username}').has('isPopulated','true').out('has').valuemap()
    """)

organics = c.clean_node(c.res[0])
organics

{'volume': -1,
 'name': 'Organic',
 'objid': '5788906859522',
 'max_volume': 1040,
 'description': 'bilogical material that can be consumed by pops',
 'replenish_rate': 10,
 'username': 'Billmanserver',
 'objtype': 'resource',
 'id': '5788906859522'}

Now that the resources are gone, the population should start to starve. 

In [51]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

0 pops will starve in nan


Series([], dtype: float64)

In [52]:
health_query =f"""
    g.V().has('objid','{planet['objid']}').as('location').in('inhabits')
        .haslabel('pop').as('pop')
        .out('isOf').as('species')
        .path()
            .by(valueMap('objid','name'))
            .by(valueMap('name','objid','health','username'))
            .by(valueMap('name','objid','consumes'))
"""

In [53]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

""


Run the following cells again and again to watch the health go down. 

In [32]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)

7 pops will starve in 1425359096973


0    None
dtype: object

In [33]:
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

,name,objid,health,username
0,[Aythakorei Or],[9427021644478],[0.5999999999999999],[Billmanserver]
1,[Aythakorei Rezche],[8742217486310],[0.5999999999999999],[Billmanserver]
2,[Aythakorei Burghuatoucal],[2128380883047],[0.5999999999999999],[Billmanserver]
3,[Aythakorei Ton],[6992584860189],[0.5999999999999999],[Billmanserver]
4,[Borgantasga Nak],[7444602298651],[0.5999999999999999],[Billmanserver]
5,[Borgantasga Bin],[7852618490329],[0.5999999999999999],[Billmanserver]
6,[Aythakorei Ziburgdiadan],[3415888692185],[0.5999999999999999],[Billmanserver]


In [46]:
consumption_df.loc[consumption_df['location_id']==planet['objid']].apply(lambda x: consumption.lower_health(c,params,x),axis=1)
c.run_query(health_query)
out = c.res
pd.DataFrame([i['objects'][1] for i in out])

7 pops will starve in 1425359096973


""
